# Απλός Ταξινομητής Εικόνων

Αυτό το σημειωματάριο σας δείχνει πώς να ταξινομήσετε εικόνες χρησιμοποιώντας ένα προεκπαιδευμένο νευρωνικό δίκτυο.

**Τι θα μάθετε:**
- Πώς να φορτώσετε και να χρησιμοποιήσετε ένα προεκπαιδευμένο μοντέλο
- Προεπεξεργασία εικόνων
- Πραγματοποίηση προβλέψεων σε εικόνες
- Κατανόηση βαθμολογιών εμπιστοσύνης

**Περίπτωση χρήσης:** Αναγνώριση αντικειμένων σε εικόνες (όπως "γάτα", "σκύλος", "αυτοκίνητο", κλπ.)

---


## Βήμα 1: Εισαγωγή Απαραίτητων Βιβλιοθηκών

Ας εισάγουμε τα εργαλεία που χρειαζόμαστε. Μην ανησυχείτε αν δεν καταλαβαίνετε όλα αυτά προς το παρόν!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Βήμα 2: Φόρτωση Προ-εκπαιδευμένου Μοντέλου

Θα χρησιμοποιήσουμε το **MobileNetV2**, ένα νευρωνικό δίκτυο που έχει ήδη εκπαιδευτεί σε εκατομμύρια εικόνες.

Αυτό ονομάζεται **Μεταφορά Μάθησης** - χρήση ενός μοντέλου που έχει εκπαιδευτεί από κάποιον άλλον!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Βήμα 3: Βοηθητικές Συναρτήσεις

Ας δημιουργήσουμε συναρτήσεις για τη φόρτωση και την προετοιμασία εικόνων για το μοντέλο μας.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Βήμα 4: Δοκιμή σε Δείγματα Εικόνων

Ας δοκιμάσουμε να ταξινομήσουμε μερικές εικόνες από το διαδίκτυο!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Κατατάξτε Κάθε Εικόνα


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Βήμα 5: Δοκιμάστε τις δικές σας εικόνες!

Αντικαταστήστε το URL παρακάτω με οποιοδήποτε URL εικόνας θέλετε να ταξινομήσετε.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Τι Μόλις Συμβαίνει;

1. **Φορτώσαμε ένα προ-εκπαιδευμένο μοντέλο** - Το MobileNetV2 εκπαιδεύτηκε σε εκατομμύρια εικόνες
2. **Προεπεξεργαστήκαμε εικόνες** - Τις αλλάξαμε μέγεθος και τις μορφοποιήσαμε για το μοντέλο
3. **Το μοντέλο έκανε προβλέψεις** - Έδωσε πιθανότητες για 1000 κατηγορίες αντικειμένων
4. **Αποκωδικοποιήσαμε τα αποτελέσματα** - Μετατρέψαμε αριθμούς σε ετικέτες κατανοητές από τον άνθρωπο

### Κατανόηση Βαθμολογιών Εμπιστοσύνης

- **90-100%**: Πολύ μεγάλη εμπιστοσύνη (σχεδόν σίγουρα σωστό)
- **70-90%**: Μεγάλη εμπιστοσύνη (πιθανότατα σωστό)
- **50-70%**: Μέτρια εμπιστοσύνη (ίσως σωστό)
- **Κάτω από 50%**: Χαμηλή εμπιστοσύνη (αβέβαιο)

### Γιατί μπορεί οι προβλέψεις να είναι λανθασμένες;

- **Ασυνήθιστη γωνία ή φωτισμός** - Το μοντέλο εκπαιδεύτηκε σε τυπικές φωτογραφίες
- **Πολλαπλά αντικείμενα** - Το μοντέλο αναμένει ένα κύριο αντικείμενο
- **Σπάνια αντικείμενα** - Το μοντέλο γνωρίζει μόνο 1000 κατηγορίες
- **Χαμηλής ποιότητας εικόνα** - Θολές ή pixelated εικόνες είναι πιο δύσκολες


## 🚀 Επόμενα Βήματα

1. **Δοκιμάστε διαφορετικές εικόνες:**
   - Βρείτε εικόνες στο [Unsplash](https://unsplash.com)
   - Δεξί κλικ → "Αντιγραφή διεύθυνσης εικόνας" για να πάρετε το URL

2. **Πειραματιστείτε:**
   - Τι συμβαίνει με την αφηρημένη τέχνη;
   - Μπορεί να αναγνωρίσει αντικείμενα από διαφορετικές γωνίες;
   - Πώς χειρίζεται πολλαπλά αντικείμενα;

3. **Μάθετε περισσότερα:**
   - Εξερευνήστε [μαθήματα Υπολογιστικής Όρασης](../lessons/4-ComputerVision/README.md)
   - Μάθετε πώς να εκπαιδεύσετε τον δικό σας ταξινομητή εικόνων
   - Κατανοήστε πώς λειτουργούν τα CNNs (Συνελικτικά Νευρωνικά Δίκτυα)

---

## 🎉 Συγχαρητήρια!

Μόλις δημιουργήσατε έναν ταξινομητή εικόνων χρησιμοποιώντας ένα υπερσύγχρονο νευρωνικό δίκτυο!

Η ίδια τεχνική χρησιμοποιείται για:
- Google Photos (οργάνωση των φωτογραφιών σας)
- Αυτοκινούμενα αυτοκίνητα (αναγνώριση αντικειμένων)
- Ιατρική διάγνωση (ανάλυση ακτινογραφιών)
- Έλεγχο ποιότητας (εντοπισμός ελαττωμάτων)

Συνεχίστε να εξερευνάτε και να μαθαίνετε! 🚀



---

**Αποποίηση ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε προσπάθειες για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτόματες μεταφράσεις ενδέχεται να περιέχουν λάθη ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
